In [1]:
!which python

/mnt/sda1/storage/miniconda/envs/ibis/bin/python


In [2]:
# import altair as alt
import ibis
import pymapd
import pandas as pd
# import jupyterlab_omnisci

/mnt/sda1/storage/miniconda/envs/ibis/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
pymapd.__version__

'0.5.1'

In [4]:
conf_ibis = dict(
    host='metis.mapd.com', user='mapd', password='HyperInteractive',
    port=443, database='mapd', protocol= 'https'
)

conn = ibis.mapd.connect(
    **conf_ibis
)

conf_pymapd = conf_ibis.copy()
conf_pymapd['dbname'] = conf_pymapd['database']
del conf_pymapd['database']

pyconn = pymapd.connect(**conf_pymapd)

In [5]:
t = conn.table('tweets_nov_feb')
s = conn.get_schema('tweets_nov_feb')

t1 = ibis.table(s)

In [6]:
n = ibis.literal(10)
min_ = t1.goog_x.min()
max_ = t1.goog_x.max()
size = (min_ - max_) // n

e = t1[((t1.goog_x - min_) / size).name('e')]
ibis.postgres.compile(e).compile().string

'SELECT (t0.goog_x - min(t0.goog_x) OVER (ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)) / floor((min(t0.goog_x) OVER (ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) - max(t0.goog_x) OVER (ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)) / :param_1) AS e \nFROM unbound_table_0 AS t0'

In [50]:
min_ = ibis.literal(t.goog_x.min().execute(), 'float64')
max_ = ibis.literal(t.goog_x.max().execute(), 'float64')

size = ((min_ - max_) / n)
e = ((t.goog_x - min_) // size)

print(e.execute().head())

Exception: Exception: Inconsistent return type for FLOOR: SELECT floor(("goog_x" - -19793718.0) / ((-19793718.0 - 19926188.0) / 10)) AS tmp
FROM tweets_nov_feb
LIMIT 10000

In [20]:
min_ = t.goog_x.min().execute()
max_ = t.goog_x.max().execute()
n = ibis.literal(10)
size = ((min_ - max_) / n)

e = ((t.goog_x - min_) / size)

print(e.execute())

0      -3.595394
1      -2.277757
2      -2.566409
3      -2.737515
4      -8.455523
5      -1.532111
6      -2.596379
7      -4.949301
8      -3.671729
9      -4.722198
10     -8.078065
11     -2.086295
12     -3.571725
13     -2.660667
14     -3.346945
15     -1.555838
16     -2.693161
17     -2.743864
18     -7.835971
19     -1.704076
20     -7.817072
21     -2.821742
22     -2.170887
23     -1.580851
24     -2.443601
25     -3.774086
26     -3.548451
27     -8.357807
28     -4.756472
29     -3.805510
          ...   
9970   -3.553902
9971   -8.373539
9972   -4.913478
9973   -3.357421
9974   -8.389025
9975   -2.946841
9976   -3.206526
9977   -1.671272
9978   -2.613703
9979   -2.916012
9980   -8.482440
9981   -3.753255
9982   -8.376708
9983   -3.277030
9984   -3.108815
9985   -2.644029
9986   -5.458760
9987   -2.558396
9988   -2.905595
9989   -3.330640
9990   -2.801528
9991   -2.508661
9992   -3.004636
9993   -3.642349
9994   -3.673183
9995   -3.073732
9996   -6.094761
9997   -3.3405

In [101]:
cursor = pyconn.execute('''
    select
      goog_x,
      goog_y,
      (goog_x - goog_y) AS tmp1,
      ROUND(goog_x / 100, 2) AS tmp2
    from tweets_nov_feb
    order by rowid
    limit 10
''')

col_names = [c.name for c in cursor.description]
result = pd.DataFrame(cursor.fetchall(), columns=col_names)
result.head()

,goog_x,goog_y,tmp1,tmp2
0,-5.495324e+06,-2365806.25,-3129517.25,-54953.234375
1,-1.083023e+05,6701961.50,-6810264.00,-1083.020142
2,-1.556940e+05,6603998.50,-6759692.50,-1556.940063
3,-1.004463e+07,3803047.75,-13847677.00,-100446.289062
4,3.451058e+06,6877843.50,-3426785.25,34510.582031


In [112]:
cursor = pyconn.execute('''
    SELECT 
      ((goog_x - goog_y) / -2.0) AS tmp1,
      (1564758.625) AS tmp2
    FROM tweets_nov_feb
    order by rowid
    limit 1
''')

col_names = [c.name for c in cursor.description]
result = pd.DataFrame(cursor.fetchall(), columns=col_names)
result.head()

,tmp1,tmp2
0,1564758.625,1564758.625


In [113]:
cursor.description

[Description(name='tmp1', type_code=3, display_size=None, internal_size=None, precision=None, scale=None, null_ok=True),
 Description(name='tmp2', type_code=4, display_size=None, internal_size=None, precision=None, scale=None, null_ok=True)]